<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_streaming_exercises_erike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=5d042c9d7b32a6c8e803c3c971c60ae63a8599042204f5743a99e93d30234863
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Dados

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Path:**  
data/ad_action_exercises

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [ ]:
# Criando um cluster local com 2 workers, 1 cores por worker e 3GB de RAM por worker

spark = SparkSession.builder\
    .master('local-cluster[2, 1, 3072]')\
    .getOrCreate()
spark

In [ ]:
AD_ACTION_CSV_PATH = "drive/MyDrive/Data/ad_action_exercises"

In [ ]:
# Stop spark session
# spark.stop()

## Foi utilizado o notebook split_ad_action_data.ipynb para criar os dados do streaming escrevendo arquivos CSV na pasta `data/ad_action_exercises`

## Resposta Atividade 1

Qual é a quantidade total de clicks e a quantidade total views dos últimos 10 segundos? Calcular a cada 10 segundos.

**Resposta:**  
```
                                       window action  count
0  (2023-09-01 02:42:10, 2023-09-01 02:42:20)  click  53452
1  (2023-09-01 02:42:30, 2023-09-01 02:42:40)   view   2544
2  (2023-09-01 02:42:00, 2023-09-01 02:42:10)   view  25630
3  (2023-09-01 02:42:00, 2023-09-01 02:42:10)  click  59280
4  (2023-09-01 02:42:20, 2023-09-01 02:42:30)  click  59627
5  (2023-09-01 02:42:10, 2023-09-01 02:42:20)   view  22961
6  (2023-09-01 02:42:30, 2023-09-01 02:42:40)  click   5946
7  (2023-09-01 02:42:20, 2023-09-01 02:42:30)   view  25273
```
Se somar o count de view dos intervalos nós devemos ter 76408 eventos

Se somar o count de click dos intervalos nós devemos ter 178305 eventos

A ordem dos registros pode variar. Não foi solicitada nenhuma ordenação na questão

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream\
    .groupBy(
        F.window('timestamp', '10 seconds', '10 seconds'),
        'action'
    ).count()\

def foreach_batch_function(df, epoch_id):
    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('update')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(40)

0
                                       window action  count
0  (2023-09-01 02:42:10, 2023-09-01 02:42:20)  click  53452
1  (2023-09-01 02:42:30, 2023-09-01 02:42:40)   view   2544
2  (2023-09-01 02:42:00, 2023-09-01 02:42:10)   view  25630
3  (2023-09-01 02:42:00, 2023-09-01 02:42:10)  click  59280
4  (2023-09-01 02:42:20, 2023-09-01 02:42:30)  click  59627
5  (2023-09-01 02:42:10, 2023-09-01 02:42:20)   view  22961
6  (2023-09-01 02:42:30, 2023-09-01 02:42:40)  click   5946
7  (2023-09-01 02:42:20, 2023-09-01 02:42:30)   view  25273


False

In [ ]:
# Stop job
query.stop()

## Resposta Atividade 2

Quais são os top 3 anúncios mais clicados dos últimos 20 segundos? Calcule a cada 15 segundos

**Resposta:**  
```
                                        window     adId  count
0   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_06   3400
1   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_07   3342
2   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_02   3225
3   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_06  12955
4   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_07  12419
5   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_02  12198
6   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_06  10203
7   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_08   9480
8   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_07   9402
9   (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_06    662
10  (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_08    644
11  (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_03    643
```

A função `foreach_batch_function` é registrada como um manipulador para cada lote de dados que chega ao final de um microbatch durante o processo de streaming. Isso permite executar operações de processamento personalizadas em cada lote de dados à medida que são processados pelo Spark Streaming.

1. Argumentos da Função:

> * df: Este é o DataFrame que contém os dados do lote atual (batch). É um DataFrame do Spark que representa os dados que chegaram durante o intervalo de tempo especificado.


> * epoch_id: Este é o identificador único do lote atual. Cada vez que a função é chamada para processar um lote, ela recebe um novo epoch_id.

2. Processamento do DataFrame:

> * Dentro da função, o DataFrame df é processado para encontrar os três principais anúncios (adId) em cada janela de tempo.
> * Um Window é criado para particionar os dados por window.start e ordená-los pela contagem decrescente de cliques (count). Isso é feito para garantir que os três principais anúncios sejam selecionados corretamente dentro de cada janela de tempo.
> * A função row_number() é usada para atribuir um número de linha a cada registro dentro de cada janela de tempo, com base na ordenação definida na janela.
> * Em seguida, o DataFrame é filtrado para manter apenas os três principais registros (anúncios) dentro de cada janela de tempo, utilizando a coluna de classificação atribuída anteriormente.
> * Finalmente, o DataFrame é ordenado novamente pela janela de tempo para garantir que os resultados sejam exibidos em ordem cronológica.


In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

# ESCREVA SEU CÓDIGO AQUI
inputStream = inputStream\
    .where(F.col('action')=='click')\
    .groupBy(
        F.window('timestamp', '20 seconds', '15 seconds'),
        'adId'
    ).count()


def foreach_batch_function(df, epoch_id):
    window_group = Window.partitionBy('window.start')\
        .orderBy(F.desc('count'))
    df = df.withColumn('rank', F.row_number().over(window_group))\
        .where(F.col('rank') <= 3)\
        .drop('rank')\
        .orderBy(F.asc('window.start'))
    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(40)

0
                                        window     adId  count
0   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_06   3400
1   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_07   3342
2   (2023-09-01 02:41:45, 2023-09-01 02:42:05)  adId_02   3225
3   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_06  12955
4   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_07  12419
5   (2023-09-01 02:42:00, 2023-09-01 02:42:20)  adId_02  12198
6   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_06  10203
7   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_08   9480
8   (2023-09-01 02:42:15, 2023-09-01 02:42:35)  adId_07   9402
9   (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_06    662
10  (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_08    644
11  (2023-09-01 02:42:30, 2023-09-01 02:42:50)  adId_03    643


False

In [ ]:
# Stop job
query.stop()

## Resposta Atividade 3

Qual é o usuário que tem mais eventos dos últimos 15 segundos? Calcule a cada 5 segundos.

**Resposta:**  
```
                                       window                               user_id  count
0  (2023-09-01 02:41:50, 2023-09-01 02:42:05)  c6ae5c85-c423-49f0-8e26-de11d4eea06a     23
1  (2023-09-01 02:41:55, 2023-09-01 02:42:10)  c6ae5c85-c423-49f0-8e26-de11d4eea06a     43
2  (2023-09-01 02:42:00, 2023-09-01 02:42:15)  c6ae5c85-c423-49f0-8e26-de11d4eea06a     57
3  (2023-09-01 02:42:05, 2023-09-01 02:42:20)  294d49e9-793a-4de7-b36e-9f07df17896a     41
4  (2023-09-01 02:42:10, 2023-09-01 02:42:25)  49822c4b-1de3-499d-ad72-c13ad9c1b35d     35
5  (2023-09-01 02:42:15, 2023-09-01 02:42:30)  ae6ff16a-a693-4894-96ae-5c514e35a334     54
6  (2023-09-01 02:42:20, 2023-09-01 02:42:35)  ae6ff16a-a693-4894-96ae-5c514e35a334     50
7  (2023-09-01 02:42:25, 2023-09-01 02:42:40)  0e965fef-50f7-4036-a258-2dfad8fa7648     31
8  (2023-09-01 02:42:30, 2023-09-01 02:42:45)  766ecd74-f9cd-4e1d-b5ea-05baafe52468      8
```

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)


inputStream = inputStream\
    .groupBy(
        F.window('timestamp', '15 seconds', '5 seconds'),
        'user_id'
    ).count()


def foreach_batch_function(df, epoch_id):
    window_group = Window.partitionBy('window.start')\
        .orderBy(F.desc('count'))
    df = df.withColumn('rank', F.row_number().over(window_group))\
        .where(F.col('rank') <= 1)\
        .drop('rank')\
        .orderBy(F.asc('window.start'))
    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(40)

0


False

                                       window  \
0  (2023-09-01 02:41:50, 2023-09-01 02:42:05)   
1  (2023-09-01 02:41:55, 2023-09-01 02:42:10)   
2  (2023-09-01 02:42:00, 2023-09-01 02:42:15)   
3  (2023-09-01 02:42:05, 2023-09-01 02:42:20)   
4  (2023-09-01 02:42:10, 2023-09-01 02:42:25)   
5  (2023-09-01 02:42:15, 2023-09-01 02:42:30)   
6  (2023-09-01 02:42:20, 2023-09-01 02:42:35)   
7  (2023-09-01 02:42:25, 2023-09-01 02:42:40)   
8  (2023-09-01 02:42:30, 2023-09-01 02:42:45)   

                                user_id  count  
0  c6ae5c85-c423-49f0-8e26-de11d4eea06a     23  
1  c6ae5c85-c423-49f0-8e26-de11d4eea06a     43  
2  c6ae5c85-c423-49f0-8e26-de11d4eea06a     57  
3  294d49e9-793a-4de7-b36e-9f07df17896a     41  
4  49822c4b-1de3-499d-ad72-c13ad9c1b35d     35  
5  ae6ff16a-a693-4894-96ae-5c514e35a334     54  
6  ae6ff16a-a693-4894-96ae-5c514e35a334     50  
7  0e965fef-50f7-4036-a258-2dfad8fa7648     31  
8  766ecd74-f9cd-4e1d-b5ea-05baafe52468      8  


In [ ]:
# Stop job
query.stop()

## Resposta Atividade 4

Qual campanha teve a maior diminuição de eventos em relação ao segundo anterior? Calcule de segundo em segundo.

**Resposta:**
```
                                        window     campaignId   diff
0   (2023-09-01 02:42:00, 2023-09-01 02:42:01)  campaignId_01   17.0
1   (2023-09-01 02:42:01, 2023-09-01 02:42:02)  campaignId_01   33.0
2   (2023-09-01 02:42:02, 2023-09-01 02:42:03)  campaignId_01   41.0
3   (2023-09-01 02:42:03, 2023-09-01 02:42:04)  campaignId_01   62.0
4   (2023-09-01 02:42:04, 2023-09-01 02:42:05)  campaignId_03  105.0
5   (2023-09-01 02:42:05, 2023-09-01 02:42:06)  campaignId_01   85.0
6   (2023-09-01 02:42:06, 2023-09-01 02:42:07)  campaignId_03   63.0
7   (2023-09-01 02:42:07, 2023-09-01 02:42:08)  campaignId_02  139.0
8   (2023-09-01 02:42:08, 2023-09-01 02:42:09)  campaignId_03  119.0
9   (2023-09-01 02:42:09, 2023-09-01 02:42:10)  campaignId_01   14.0
10  (2023-09-01 02:42:10, 2023-09-01 02:42:11)  campaignId_02   79.0
11  (2023-09-01 02:42:11, 2023-09-01 02:42:12)  campaignId_03   92.0
12  (2023-09-01 02:42:12, 2023-09-01 02:42:13)  campaignId_01   92.0
13  (2023-09-01 02:42:13, 2023-09-01 02:42:14)  campaignId_03   74.0
14  (2023-09-01 02:42:14, 2023-09-01 02:42:15)  campaignId_01   57.0
15  (2023-09-01 02:42:16, 2023-09-01 02:42:17)  campaignId_01  111.0
16  (2023-09-01 02:42:17, 2023-09-01 02:42:18)  campaignId_02  103.0
17  (2023-09-01 02:42:18, 2023-09-01 02:42:19)  campaignId_01   39.0
18  (2023-09-01 02:42:19, 2023-09-01 02:42:20)  campaignId_03   60.0
19  (2023-09-01 02:42:20, 2023-09-01 02:42:21)  campaignId_02   32.0
20  (2023-09-01 02:42:21, 2023-09-01 02:42:22)  campaignId_02  148.0
21  (2023-09-01 02:42:22, 2023-09-01 02:42:23)  campaignId_03   79.0
22  (2023-09-01 02:42:23, 2023-09-01 02:42:24)  campaignId_03   51.0
23  (2023-09-01 02:42:24, 2023-09-01 02:42:25)  campaignId_01   24.0
24  (2023-09-01 02:42:25, 2023-09-01 02:42:26)  campaignId_02   81.0
25  (2023-09-01 02:42:26, 2023-09-01 02:42:27)  campaignId_03  129.0
26  (2023-09-01 02:42:27, 2023-09-01 02:42:28)  campaignId_01  103.0
27  (2023-09-01 02:42:28, 2023-09-01 02:42:29)  campaignId_03  209.0
28  (2023-09-01 02:42:29, 2023-09-01 02:42:30)  campaignId_02   97.0
29  (2023-09-01 02:42:30, 2023-09-01 02:42:31)  campaignId_02    NaN
```

In [ ]:
from pyspark.sql.functions import lead

inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream\
    .groupBy(
        F.window('timestamp', '1 seconds', '1 seconds'), 'campaignId'
    ).count().orderBy('window') # O DataFrame resultante é ordenado pela coluna de janela de tempo.

def foreach_batch_function(df, epoch_id):
    windowSpec  = Window.partitionBy("campaignId").orderBy("window.start")

    df = df.withColumn("lead", lead("count", 1).over(windowSpec))
    df = df.withColumn("diff", F.col('count') - F.col('lead'))

    windowDept = Window.partitionBy("window.start").orderBy(F.col("diff").desc())
    df = df.withColumn("row", F.row_number().over(windowDept)) \
    .filter(F.col("row") <= 1) \
    .orderBy(F.asc('window.start'))

    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(50)

0
                                        window     campaignId  count    lead   diff  row
0   (2023-09-01 02:42:00, 2023-09-01 02:42:01)  campaignId_01   2643  2626.0   17.0    1
1   (2023-09-01 02:42:01, 2023-09-01 02:42:02)  campaignId_01   2626  2593.0   33.0    1
2   (2023-09-01 02:42:02, 2023-09-01 02:42:03)  campaignId_01   2593  2552.0   41.0    1
3   (2023-09-01 02:42:03, 2023-09-01 02:42:04)  campaignId_01   2552  2490.0   62.0    1
4   (2023-09-01 02:42:04, 2023-09-01 02:42:05)  campaignId_03   2898  2793.0  105.0    1
5   (2023-09-01 02:42:05, 2023-09-01 02:42:06)  campaignId_01   2623  2538.0   85.0    1
6   (2023-09-01 02:42:06, 2023-09-01 02:42:07)  campaignId_03   2905  2842.0   63.0    1
7   (2023-09-01 02:42:07, 2023-09-01 02:42:08)  campaignId_02   3065  2926.0  139.0    1
8   (2023-09-01 02:42:08, 2023-09-01 02:42:09)  campaignId_03   2938  2819.0  119.0    1
9   (2023-09-01 02:42:09, 2023-09-01 02:42:10)  campaignId_01   2591  2577.0   14.0    1
10  (2023-09-01 02:

False

In [ ]:
# Stop job
query.stop()

## Resposta Atividade 5

Qual é a porcentagem de clicks dado o total de eventos dos últimos 12 segundos? Calcule a cada 6 segundos.

**Resposta**  
```
                                       window  percentage
0  (2023-09-01 02:41:54, 2023-09-01 02:42:06)       69.84
1  (2023-09-01 02:42:00, 2023-09-01 02:42:12)       69.84
2  (2023-09-01 02:42:06, 2023-09-01 02:42:18)       70.00
3  (2023-09-01 02:42:12, 2023-09-01 02:42:24)       69.91
4  (2023-09-01 02:42:18, 2023-09-01 02:42:30)       70.08
5  (2023-09-01 02:42:24, 2023-09-01 02:42:36)       70.42
6  (2023-09-01 02:42:30, 2023-09-01 02:42:42)       70.04
```

Função `foreach_batch_function()`:


1. `df = df.withColumn('total_events', F.col('count')+F.lag(F.col('count'), 1).over(windowSpec))`:
- Nesta linha, estamos calculando o total de eventos para cada janela de tempo (window). Isso é feito somando a contagem atual de eventos (count) com o valor da contagem de eventos na linha anterior dentro da mesma janela de tempo. Para isso, usamos a função lag do Spark para acessar o valor anterior da contagem de eventos dentro da janela. O resultado é armazenado na coluna total_events.

2. `df = df.withColumn("total_events", F.last('total_events', True).over(windowSpec))`:
- Aqui, estamos preenchendo os valores null na coluna total_events com o último valor não nulo encontrado dentro da mesma janela de tempo. Isso é feito usando a função last do Spark com o argumento True, que especifica para ignorar os valores null. O resultado é que todas as linhas dentro da mesma janela de tempo terão o mesmo valor para total_events, que representa o total de eventos dentro dessa janela.

In [ ]:
inputStream = spark.readStream.csv(
    AD_ACTION_CSV_PATH,
    schema="timestamp TIMESTAMP, \
        user_id STRING, \
        action STRING, \
        adId STRING, \
        campaignId STRING"
)

inputStream = inputStream\
    .groupBy(
        F.window('timestamp', '12 seconds', '6 seconds'),
        'action'
    ).count()

def foreach_batch_function(df, epoch_id):
    windowSpec  = Window.partitionBy("window").orderBy("window.start")

    df = df.withColumn('total_events', F.col('count')+F.lag(F.col('count'), 1).over(windowSpec))
    df = df.withColumn("total_events", F.last('total_events', True).over(windowSpec))
    df = df.where(F.col('action') == 'click')
    df = df.withColumn('percent', F.round(100*(F.col('count')/F.col('total_events')), 2))
    df = df.orderBy(F.asc('window.start'))

    print(epoch_id)
    print(df.toPandas())

query = inputStream\
    .writeStream\
    .outputMode('complete')\
    .foreachBatch(foreach_batch_function)\
    .start()
query.awaitTermination(40)

0
                                       window action  count  total_events  percent
0  (2023-09-01 02:41:54, 2023-09-01 02:42:06)  click  35580         50946    69.84
1  (2023-09-01 02:42:00, 2023-09-01 02:42:12)  click  71165        101892    69.84
2  (2023-09-01 02:42:06, 2023-09-01 02:42:18)  click  65377         93397    70.00
3  (2023-09-01 02:42:12, 2023-09-01 02:42:24)  click  65289         93391    69.91
4  (2023-09-01 02:42:18, 2023-09-01 02:42:30)  click  71402        101880    70.08
5  (2023-09-01 02:42:24, 2023-09-01 02:42:36)  click  41851         59430    70.42
6  (2023-09-01 02:42:30, 2023-09-01 02:42:42)  click   5946          8490    70.04


False

In [ ]:
# Stop job
query.stop()